In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

tree = Phylo.read('tree.nwk', 'newick')
print tree


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(tree)? (<ipython-input-1-a4b2d514a864>, line 6)

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

tree = Phylo.read('tree.nwk', 'newick')
Phylo.draw(tree)

NameError: name 'Phylo' is not defined

In [8]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Phylo
from Bio.Cluster import treecluster
%matplotlib inline

# Create tree
tree = Phylo.read('tree.nwk', 'newick')
tree.ladderize() 
Phylo.draw_ascii(tree, file=None, column_width =125)


# Select one representative of each cluster - gene is dtermined using BLAST
record_dict =  SeqIO.to_dict(SeqIO.parse('seqs.aligned.fa','fasta'))
cluster_list = ['hu.39','pi.2','hu.17','rh.2','rh.40','rh.43','hu.14','hu.43','cy.2','rh.35','cy.3']
"""
## Cluster 1 - Adeno-associated virus isolate hu.39 capsid protein VP1 (cap) gene, complete cds
print(record_dict['hu.39'].format('fasta'))
## Cluster 2 - 
print(record_dict['pi.2'].format('fasta'))
## Cluster 3 - hu.17
print(record_dict['hu.39'].format('fasta'))
"""
"""for specimen in cluster_list:
    print(record_dict[specimen].format('fasta'))
"""
seq = record_dict['hu.39'].format('fasta')
AT = (seq.count('A')+seq.count('T'))/len(seq)
CG = 1 - AT
check = (seq.count('C')+seq.count('G'))/len(seq)
print(check + AT)
seq.information_content(self, start=0, end=None, e_freq_table=None, log_base=2, chars_to_ignore=None, pseudo_count=0)

 , hu.39
 |
 | rh.50
 |
_, rh.49
 |
 |_ rh.57
 |
 | rh.51
 |
 , rh.53
 |
 | rh.64
 |
 , rh.52
 |
 , rh.61
 |
 | __ rh.58
 ||
 ||                                                    , pi.2
 ||                  __________________________________|
 ||                 |                                  |_ pi.3
 ||                 |                                  |
 ||                 |                                  | pi.1
 ||                 |
 ||                 |                   __ hu.17
 ||              ___|                 _|
 ||             |   |                | | hu.6
 ||             |   |    ____________|
 ||             |   |   |            |  , rh.10
 ||             |   |   |            |__|
  |             |   |   |               |, bb.1
  |             |   |___|               ||
  |             |       |                | bb.2
  |             |       |
  |             |       |       ___________ rh.2
  |             |       |      |
  |             |       |______|        

AttributeError: 'str' object has no attribute 'information_content'